In [1]:
!pip install -U -q preprocessor
!pip install -U -q vaderSentiment
!pip install -U -q nltk
!pip install -U -q textblob
!pip install -U -q gensim
!pip install -U -q tqdm
!pip install -U -q bs4
!pip install python-Levenshtein
!pip install -U -q plotly

     |████████████████████████████████| 133kB 10.0MB/s 
     |████████████████████████████████| 1.5MB 7.8MB/s 
     |████████████████████████████████| 23.9MB 1.7MB/s 
     |████████████████████████████████| 81kB 4.8MB/s 
     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149820 sha256=4405d9cfe29e554fbcdbae804503546e3495aa79f2904c8452df073e27eaf12c
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 13.2MB 7.2MB/s 


In [3]:
import preprocessor as p
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import gensim
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
import plotly.io as pio
py.init_notebook_mode(connected=True)
# Required for plotly to run on colab
pio.renderers.default = 'colab'
%matplotlib inline

In [4]:
tqdm.pandas()

**Connects to your google drive**

In [6]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')

Mounted at /content/drive/


1. Dataframe with 16 M rows
2. Note - To process huge data of 3.9 GB with 16M rows of tweet data, this code was processed in Google Cloud machines with GPU
3. Drop useless columns

In [7]:

# df = pd.read_csv("tweets-bulk.csv", sep=';', nrows= 12143600, header=0, error_bad_lines=False) 
# 100,000 rows to test the processing with limited data set
df = pd.read_csv("tweet_1L.csv")
data = df.drop(["url","fullname"],axis=1)


In [ ]:
data.head(2)
data.shape

(100000, 8)

**Clean function using a combination of re and preprocessor**

In [8]:
def cleaning(text):
    cleantext = p.clean(text)
    cleantext = re.sub('[^a-zA-Z ]','',cleantext)
    return cleantext
def clean(data):
    #cleanr = re.compile("<.*?>")
    cleanr= re.sub(r"http\S+", "", data)
    
    cleanr= re.sub(r"pic.\S+", "", cleanr)
    #cleanr= re.sub(cleanr, "", data)
    #cleantext = re.sub(cleanr,'',data)
    cleantext = re.sub('[^a-zA-Z# ]','',cleanr)
    clean_hash = re.sub(r"#\S+","",cleantext)
    
    return clean_hash
data["text"] = data.progress_apply(lambda x:clean(str(x["text"])),axis=1)

data.head(2)

100%|██████████| 100000/100000 [00:02<00:00, 41256.56it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text
0,0,1132977055300300800,KamdemAbdiel,2019-05-27 11:49:14+00,0,0,0,appena uscito un nuovo video LES CRYPTOMONNAI...
1,1,1132977073402736640,bitcointe,2019-05-27 11:49:18+00,0,0,0,Cardano Digitize Currencies EOS ROI ATampT B...


**Tokenize and Stem and then join the text**

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
porter = PorterStemmer()

def stemSentence(sentence):
    #Tokenize words list
    tokens = [word.lower() for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    stem_sentence=[]
    # Create list of word tokens after removing the stopwords
    filtered_sentence =[] 
    for word in tokens:
      if word not in stopwords:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**TextBlob functions to calculate Sentiment scores**

In [11]:
data["subjectivity"] = np.zeros((data.shape[0],1))
data["blob_sent"] = np.zeros((data.shape[0],1))
def textblob_sent(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[0]
def textblob_sub(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[1]

In [12]:
print(data["text"][6])
print(textblob_sent(data["text"][6]))
print(textblob_sent(stemSentence(data["text"][6])))

nwoodfine We have been building on the real  SVWhat have you been building on broken BTC
-0.1
-0.1


**Calculating blob sentiment for every tweet**

In [13]:
data["blob_sent"] = data.progress_apply(lambda x:textblob_sent(str(x["text"])),axis=1)

100%|██████████| 100000/100000 [01:17<00:00, 1283.64it/s]


**Save the results to save time for processing again**

In [ ]:
data.to_csv("blob_sent_recorded_1L.csv")
data.head(10)

,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent
0,0,1132977055300300800,KamdemAbdiel,2019-05-27 11:49:14+00,0,0,0,appena uscito un nuovo video LES CRYPTOMONNAI...,0.0,0.0
1,1,1132977073402736640,bitcointe,2019-05-27 11:49:18+00,0,0,0,Cardano Digitize Currencies EOS ROI ATampT B...,0.0,0.0
2,2,1132977023893139456,3eyedbran,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasnt caught in the st...,0.0,0.0
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.0,0.0
4,4,1132977092340191232,mmursaleen72,2019-05-27 11:49:23+00,0,0,0,Spiv Nosar Baz BITCOIN Is An Asset amp NOT A C...,0.0,0.0
5,5,1132977100309319680,0nurTOKA,2019-05-27 11:49:25+00,0,0,0,inceldii yerden kopsun bakalm ye kadar bir h...,0.0,0.0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.0,-0.1
7,7,1132977108836323330,jabur_guilherme,2019-05-27 11:49:27+00,0,0,0,pedronauck como investidor vc um timo dev Sor...,0.0,0.0
8,8,1132977131850350593,INTBICON,2019-05-27 11:49:32+00,0,0,0,,0.0,0.0
9,9,1132977132714561536,MLWright15,2019-05-27 11:49:32+00,0,0,0,CHANGE IS COMINGGET READY Boom Another CB Jab ...,0.0,0.0


**Subjectivity calculation for each tweet**

In [14]:
data["subjectivity"] = data.progress_apply(lambda x:textblob_sub(x["text"]),axis=1)
data.to_csv("subjectivity_recorded_1L.csv")


100%|██████████| 100000/100000 [01:18<00:00, 1278.89it/s]


In [ ]:
data.shape

(100000, 10)

**Drop rows with zero sentiment and sbjectivity**

In [15]:
index_names = data[ (data['subjectivity'] == 0.0) & (data['blob_sent'] == 0.000000)].index
  
# drop these given row indexes from dataFrame
data.drop(index_names, inplace = True)  

In [ ]:
data.shape

(39652, 10)

**Vader sentiment**

In [36]:
sid = SentimentIntensityAnalyzer()
def vader(text):
    score = sid.polarity_scores(text)
    return score["compound"]


**VADER sentiment calculation**

In [37]:
data["vader_sent"] = data.progress_apply(lambda x:vader(str(x["text"])),axis=1)
data.head(5)

100%|██████████| 39652/39652 [00:07<00:00, 5060.45it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,ad_count,adwords,ad_class,prominent_user,key_count,keywords,RT_score,fav_score,date,time,hour_score,daily_score,Daily Weight,hourly Weight,vader_sent
time,,,,,,,,,,,,,,,,,,,,,,,,,
2019-05-27 11:49:22,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.400000,0.000000,0,[],1,0,0,[],0.000000,0.000000,2019-05-27 11:49:22,2019-05-27 11:49:22,1,0,0.000000,1.000000,0.0000
2019-05-27 11:49:25,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.350000,-0.000000,0,[],1,0,0,[],0.000000,0.000000,2019-05-27 11:49:25,2019-05-27 11:49:25,1,0,0.000000,1.000000,-0.4767
2019-05-27 08:13:06,14,1132922663113023489,CCNMarkets,2019-05-27 08:13:06+00,5,167,68,Bitcoin Price Hits in New High Whats Driving...,0.497273,12.919815,0,[],1,0,0,[],4.219508,5.117994,2019-05-27 08:13:06,2019-05-27 08:13:06,0,0,9.337502,9.337502,0.0000
2019-05-27 11:49:17,17,1132977067048525824,malfouh,2019-05-27 11:49:17+00,1,1,1,The QLC airdrop tokens for every QLC holder...,0.850000,0.000000,1,[airdrop],1,0,0,[],0.000000,0.000000,2019-05-27 11:49:17,2019-05-27 11:49:17,1,0,0.000000,1.000000,0.0000
2019-05-27 11:27:22,18,1132971551471603712,BoilingPointLTC,2019-05-27 11:27:22+00,1,19,6,You have roughly days left to get your tic...,0.200000,0.000000,0,[],1,0,0,[],1.791759,2.944439,2019-05-27 11:27:22,2019-05-27 11:27:22,0,0,4.736198,4.736198,0.0000


**Weight corresponding to adwords are calculated here**

In [17]:
adwords = ["give","referral","referal","bonus","signup","invite","freebitcoin","lucky",
          "lottery","giveaway","coupon","prizes","enter","airdrop","game","contest","bonuses"]
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')
def adchecker(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return count
def adword(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return words

**Computation of adwords weight**

In [18]:
data["ad_count"] = np.zeros((data.shape[0],1))
data["adwords"] = np.zeros((data.shape[0],1))

data["ad_count"] = data.progress_apply(lambda x:adchecker(x["text"]),axis=1)
data["adwords"] = data.progress_apply(lambda x:adword(x["text"]),axis=1)

# data.to_csv("clean100.csv")


100%|██████████| 39652/39652 [00:00<00:00, 39996.39it/s]


**Classifies whether a tweet is an ad or not**

In [19]:
def ad_classifier(x):
    try:
        if x >1 :
            return 0
        else:
            return 1
    except:
        print(x)

data["ad_class"] = data.progress_apply(lambda x:ad_classifier(int(x["ad_count"])),axis=1)

100%|██████████| 39652/39652 [00:00<00:00, 71014.31it/s]


**Checking counts**

In [20]:
data.astype({'ad_count': 'float64'}).dtypes
#nlarg = data.nlargest(30,"ad_count")
count = data["ad_class"].value_counts()
print(count)

1    39604
0       48
Name: ad_class, dtype: int64


**Weightage with promising users**

In [21]:
prominentusers = set(['iamjosephyoung', 'Bitcoin', 'BitcoinMagazine', 'btc', 'BTCNewsletter', 'BTCTN', 'coindesk', 'MoonOverlord',
                      'TuurDemeester', 'CryptoCoinsNews', 'LandM_Marius', 'kerooke', 'loomdart', 'GemCrypto', 
                      'coinnounce', 'MatiGreenspan', 'JWilliamsFstmed', 'VitalikButerin', 'VladZamfir', 'aantonop',
                      'ethereumJoseph', 'gavofyork', 'naval', 'tayvano_', 'NickSzabo4', 'cburniske',
                      'jwolpert', 'AndrewDARMACAP', 'ricburton', 'bcrypt', 'mikeraymcdonald',
                      'evan_van_ness', 'FEhrsam', 'JohnLilic', 'avsa', 'antiprosynth', 'nlw', 'brian_armstrong', 
                      'twobitidiot', 'APompliano', 'Cointelegraph', 'coinmetrics'])
def usernamer(x):
    if x in prominentusers:
        return 1
    else:
        return 0
data["prominent_user"] = data.progress_apply(lambda x:usernamer(x["user"]),axis=1)

data.head(5)


100%|██████████| 39652/39652 [00:00<00:00, 71299.95it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,ad_count,adwords,ad_class,prominent_user
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.400000,0.000000,0,[],1,0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.350000,-0.100000,0,[],1,0
14,14,1132922663113023489,CCNMarkets,2019-05-27 08:13:06+00,5,167,68,Bitcoin Price Hits in New High Whats Driving...,0.497273,0.148182,0,[],1,0
17,17,1132977067048525824,malfouh,2019-05-27 11:49:17+00,1,1,1,The QLC airdrop tokens for every QLC holder...,0.850000,0.250000,1,[airdrop],1,0
18,18,1132971551471603712,BoilingPointLTC,2019-05-27 11:27:22+00,1,19,6,You have roughly days left to get your tic...,0.200000,0.000000,0,[],1,0


**Weightage with keywords**

In [22]:
keywords = ['onchain transaction', 'congestion', 'market cap','market cap drop', 'dispersion of returns', 'downturn', 'exchange volume', 'ATH', 'ath', 'atl', 'ATL',
            'bear trap', 'btfd', 'BTFD', 'buy wall', 'deflation', 'dump', 'fomo', 'FOMO', 'mooning', 'whale']
def keychecker(text):
    words = []
    count = 0
    for i in keywords:
        if (contains_word(str(text),i)):
            count+= 1
            words.append(i)
    return count
def keyword(text):
    words = []
    count = 0
    for i in keywords:
        if (contains_word(str(text),i)):
            count+= 1
            words.append(i)
    return words
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

data["key_count"] = data.progress_apply(lambda x:keychecker(x["text"]),axis=1)
data["keywords"] = data.progress_apply(lambda x:keyword(x["text"]),axis=1)


100%|██████████| 39652/39652 [00:01<00:00, 32452.42it/s]


In [24]:
promu = data.nlargest(30,"key_count")
promu

,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,ad_count,adwords,ad_class,prominent_user,key_count,keywords
70962,70962,1127093880929832960,leb_crypto,2019-05-11 04:11:36+00,0,0,0,We may be seeing FOMO nowHowever Real FOMO kic...,0.300000,0.200000,0,[],1,0,3,"[ATH, fomo, FOMO]"
9396,9396,1126844252347039744,JohnyCrypto1,2019-05-10 11:39:40+00,0,0,0,Breaking was very easy something doesnt seem ...,0.345238,0.078571,0,[],1,0,2,"[dump, FOMO]"
12892,12892,1126857412571324418,bukshukalaka,2019-05-10 15:05:43+00,0,1,1,jonathanaird fidelity of half their investors...,0.333333,-0.083333,0,[],1,0,2,"[market cap, fomo]"
23065,23065,1126893173857554433,bitcoinization,2019-05-10 14:54:03+00,0,0,0,realKuzaG Agreed but I believe bitcoin will be...,0.677273,-0.181818,0,[],1,0,2,"[ATH, fomo]"
23826,23826,1126896492055289856,Norcalsteve22,2019-05-10 15:07:15+00,0,0,0,CNik coinbase You think the mil tether burn m...,0.666667,0.333333,0,[],1,0,2,"[dump, whale]"
33611,33611,1126928662874476551,bradmillscan,2019-05-10 17:15:05+00,0,0,0,whitebuffalo aantonop which is why I dont like...,0.330909,0.287273,0,[],1,0,2,"[fomo, FOMO]"
37026,37026,1126943287191928838,BurniteGeorge,2019-05-10 18:13:11+00,0,0,0,Honestly dont know what to dump my open intoc...,0.410417,-0.085417,0,[],1,0,2,"[ATH, dump]"
45458,45458,1126983806114762753,rickeyjohnson12,2019-05-10 20:54:12+00,0,0,0,xtdisnkfe Yea that makes a lot of sense I thin...,0.557143,0.177143,0,[],1,0,2,"[dump, FOMO]"
47837,47837,1126998115662192640,danielchiam7,2019-05-10 21:51:04+00,0,0,0,jebus Unless of course they already sold every...,0.100000,0.000000,0,[],1,0,2,"[ATH, FOMO]"
51487,51487,1127016181171310592,bukshukalaka,2019-05-10 23:02:51+00,0,0,0,To me of half fidelity trill investments is ...,0.166667,-0.166667,0,[],1,0,2,"[market cap, FOMO]"


**Weightage based on likes and retweets**

In [26]:
def log_score(x):
    try:
        #x.astype(float)
        if int(x)<1:
            return 0
        else:
            return np.log(int(x))
    except:
        print(x)
        return 0
data["RT_score"] = data.progress_apply(lambda x:log_score(x["retweets"]),axis=1)
data["fav_score"] = data.progress_apply(lambda x:log_score(x["likes"]),axis=1)
data.head(2)

100%|██████████| 39652/39652 [00:00<00:00, 79307.44it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,ad_count,adwords,ad_class,prominent_user,key_count,keywords,RT_score,fav_score
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.40,0.0,0,[],1,0,0,[],0.0,0.0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.35,-0.1,0,[],1,0,0,[],0.0,0.0


In [28]:
# Timestamp conversion to datetime without localization
data["date"] = pd.to_datetime(data["timestamp"],errors= "coerce").dt.tz_localize(None)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39652 entries, 3 to 99998
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      39652 non-null  int64         
 1   id              39652 non-null  int64         
 2   user            39652 non-null  object        
 3   timestamp       39652 non-null  object        
 4   replies         39652 non-null  int64         
 5   likes           39652 non-null  int64         
 6   retweets        39652 non-null  int64         
 7   text            39652 non-null  object        
 8   subjectivity    39652 non-null  float64       
 9   blob_sent       39652 non-null  float64       
 10  vader_sent      39652 non-null  float64       
 11  ad_count        39652 non-null  int64         
 12  adwords         39652 non-null  object        
 13  ad_class        39652 non-null  int64         
 14  prominent_user  39652 non-null  int64         
 15  ke

**Add time as index**

In [29]:
data['time'] = data['date']
data.index = data['time']


**Daily and hourly weightage method**

In [31]:
def half(duration,x):
    try:
        if duration == "daily":
            #print(x)
            if int(x) >11:
                return 1
            else:
                return 0
        if duration =="hourly":
            if int(x) > 30:
                return 1
            else:
                return 0
    except:
        print(x)
        return 0
        
data["hour_score"] = data.progress_apply(lambda x:half("hourly",x["date"].minute),axis=1)
data["daily_score"] = data.progress_apply(lambda x:half("daily",x["date"].hour),axis=1)

100%|██████████| 39652/39652 [00:01<00:00, 39412.51it/s]


**Calculate daily and hourly weights using other attributes calculated in the previous steps**

In [34]:
data["Daily Weight"] = (data["prominent_user"]+data["key_count"]+data["RT_score"]+data["fav_score"]+data["daily_score"])*data["ad_class"]
data["hourly Weight"] = (data["prominent_user"]+data["key_count"]+data["RT_score"]+data["fav_score"]+data["hour_score"])*data["ad_class"]


**Fusing daily weight and hourly weight with TextBlob and VADER respectively**

In [38]:
data["blob_sent"] = data["blob_sent"]*data["Daily Weight"]
data["vader_sent"] = data["vader_sent"]*data["hourly Weight"]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3682809 entries, 3 to 12143593
Data columns (total 23 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              object        
 1   user            object        
 2   timestamp       object        
 3   replies         float64       
 4   likes           float64       
 5   retweets        float64       
 6   text            object        
 7   subjectivity    float64       
 8   blob_sent       float64       
 9   vader_sent      float64       
 10  ad_count        int64         
 11  adwords         object        
 12  ad_class        int64         
 13  prominent_user  int64         
 14  key_count       int64         
 15  keywords        object        
 16  RT_score        float64       
 17  fav_score       float64       
 18  date            datetime64[ns]
 19  hour_score      int64         
 20  daily_score     int64         
 21  Daily Weight    float64       
 22  hourly Weight   f

**Calculate hourly features**

In [39]:
sentiment_grouped_hourly = data.groupby(pd.Grouper(key="time", freq="1h")).agg(
    Daily_Weight_mean_by_hour = ('Daily Weight', np.mean),
    Daily_Weight_count_by_hour = ('Daily Weight', 'count'),
    blob_sent_mean_by_hour = ('blob_sent', np.mean),
    subjectivity_mean_by_hour = ('subjectivity', np.mean),
    vader_sent_mean_by_hour = ('vader_sent', np.mean)
)

**Drop Null rows as data will be missing fo some hour values**

In [40]:
drop_column_subset = sentiment_grouped_hourly.columns.values[1:]
sentiment_grouped_hourly.dropna(subset=drop_column_subset, inplace=True)
sentiment_grouped_hourly.head()

,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour
time,,,,,
2012-08-02 17:00:00,1.0,1,0.30,0.100000,0.2263
2013-08-16 18:00:00,1.0,1,0.05,0.150000,0.0000
2014-02-20 17:00:00,1.0,1,0.10,1.000000,0.0000
2014-08-01 18:00:00,1.0,1,0.00,0.066667,0.0000
2014-10-03 05:00:00,0.0,1,0.00,0.066667,-0.4404


**Method to classify the movement of features**

In [41]:
def movement_classifier(x):
    try:
        if x >= 0 :
            return 1
        else:
            return 0
    except:
        print(x)

**Calculate differences to apply the movement_classifier**

In [42]:
sentiment_grouped_hourly["tweet_diff"] = sentiment_grouped_hourly["Daily_Weight_count_by_hour"].diff()
sentiment_grouped_hourly["blob_sent_mean_by_hour_diff"] = sentiment_grouped_hourly["blob_sent_mean_by_hour"].diff()
sentiment_grouped_hourly["vader_sent_mean_by_hour_diff"] = sentiment_grouped_hourly["vader_sent_mean_by_hour"].diff()
sentiment_grouped_hourly["tweet_movement"] = sentiment_grouped_hourly["tweet_diff"].progress_apply(lambda x:movement_classifier(x))
sentiment_grouped_hourly["blob_sent_movement"] = sentiment_grouped_hourly["blob_sent_mean_by_hour_diff"].progress_apply(lambda x:movement_classifier(x))
sentiment_grouped_hourly["vader_sent_movement"] = sentiment_grouped_hourly["vader_sent_mean_by_hour_diff"].progress_apply(lambda x:movement_classifier(x))

100%|██████████| 1343/1343 [00:00<00:00, 243840.11it/s]


**Save the data to save the processing time for future use**

In [43]:
sentiment_grouped_hourly.head()
sentiment_grouped_hourly.to_csv("twitter_sentiments_hourly_processed_data.csv")

In [44]:
sentiment_grouped_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1343 entries, 2012-08-02 17:00:00 to 2019-05-27 12:00:00
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Daily_Weight_mean_by_hour     1343 non-null   float64
 1   Daily_Weight_count_by_hour    1343 non-null   int64  
 2   blob_sent_mean_by_hour        1343 non-null   float64
 3   subjectivity_mean_by_hour     1343 non-null   float64
 4   vader_sent_mean_by_hour       1343 non-null   float64
 5   tweet_diff                    1342 non-null   float64
 6   blob_sent_mean_by_hour_diff   1342 non-null   float64
 7   vader_sent_mean_by_hour_diff  1342 non-null   float64
 8   tweet_movement                1343 non-null   int64  
 9   blob_sent_movement            1343 non-null   int64  
 10  vader_sent_movement           1343 non-null   int64  
dtypes: float64(7), int64(4)
memory usage: 125.9 KB
